In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import layers, models


In [ ]:
import requests
from io import BytesIO

# Load dataset
data_path = '/content/drive/MyDrive/IMDB_CSV/gt.csv'
data = pd.read_csv(data_path)

# Preprocess images
image_data = []
labels = []

# Loop through the data and preprocess images
for index, row in data.iterrows():
    img_url = row['label']
    age = row['score']

    try:
        # Load image from URL
        response = requests.get(img_url)
        img = cv2.imdecode(np.frombuffer(response.content, np.uint8), -1)

        if img is not None:
            # Resize image to a fixed size
            img = cv2.resize(img, (64, 64))

            # Convert grayscale images to RGB
            if len(img.shape) == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

            # Append image and label to lists
            image_data.append(img)
            labels.append(age)
        else:
            print(f"Error loading image: {img_url}")
    except Exception as e:
        print(f"Error downloading image: {img_url}, {e}")

# Convert lists to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

# Normalize images
image_data = image_data / 255.0



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the number of models in the ensemble
num_models = 5
models = []

# Create instances of the CNN model
for _ in range(num_models):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    models.append(model)


In [ ]:
batch_size = 32
epochs = 30

# Train each model on different subsets of the training data
for i, model in enumerate(models):
    print(f"Training Model {i + 1}")
    model.fit(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        steps_per_epoch=len(X_train) / batch_size,
        epochs=epochs,
        validation_data=(X_test, y_test)
    )


Training Model 1
Epoch 1/30
228/228 [==============================] - 71s 304ms/step - loss: 411.7746 - mae: 16.8876 - val_loss: 326.1116 - val_mae: 15.6520
Epoch 2/30
228/228 [==============================] - 69s 303ms/step - loss: 348.1898 - mae: 15.8194 - val_loss: 332.5763 - val_mae: 15.5196
Epoch 3/30
228/228 [==============================] - 69s 303ms/step - loss: 345.6618 - mae: 15.6895 - val_loss: 301.4508 - val_mae: 14.9487
Epoch 4/30
228/228 [==============================] - 69s 304ms/step - loss: 336.4678 - mae: 15.4719 - val_loss: 305.7832 - val_mae: 15.0794
Epoch 5/30
228/228 [==============================] - 74s 326ms/step - loss: 330.1015 - mae: 15.3434 - val_loss: 300.9243 - val_mae: 14.8969
Epoch 6/30
228/228 [==============================] - 70s 305ms/step - loss: 329.3355 - mae: 15.2666 - val_loss: 313.5413 - val_mae: 15.0514
Epoch 7/30
228/228 [==============================] - 69s 303ms/step - loss: 318.3592 - mae: 15.0449 - val_loss: 294.0895 - val_mae: 14.7

In [ ]:
import numpy as np

# Predict age labels for test data
y_pred = model.predict(X_test)

# Calculate mean absolute error (MAE)
mae = np.mean(np.abs(y_pred.flatten() - y_test))

# Calculate accuracy (percentage of predictions within a tolerance)
tolerance = 5  # Define tolerance for correct predictions
correct_predictions = np.sum(np.abs(y_pred.flatten() - y_test) <= tolerance)
total_predictions = len(y_test)
accuracy = (correct_predictions / total_predictions) * 100

print("Test Mean Absolute Error:", mae)
print("Test Accuracy:", accuracy, "%")


58/58 [==============================] - 4s 69ms/step
Test Mean Absolute Error: 13.818204679645476
Test Accuracy: 19.672131147540984 %


In [ ]:
# Calculate the mean absolute error (MAE)
mae = np.mean(np.abs(y_pred.flatten() - y_test))

# Print actual and adjusted predicted age labels for a few samples
num_samples = 10  # Number of samples to display
for i in range(num_samples):
    print("Sample", i+1)
    print("Actual Age:", y_test[i])
    adjusted_age = round(y_pred[i][0] + mae)  # Adjusted predicted age
    print("Adjusted Predicted Age:", adjusted_age)
    print()


Sample 1
Actual Age: 46
Adjusted Predicted Age: 46

Sample 2
Actual Age: 24
Adjusted Predicted Age: 56

Sample 3
Actual Age: 28
Adjusted Predicted Age: 48

Sample 4
Actual Age: 49
Adjusted Predicted Age: 45

Sample 5
Actual Age: 70
Adjusted Predicted Age: 50

Sample 6
Actual Age: 42
Adjusted Predicted Age: 49

Sample 7
Actual Age: 64
Adjusted Predicted Age: 58

Sample 8
Actual Age: 51
Adjusted Predicted Age: 55

Sample 9
Actual Age: 59
Adjusted Predicted Age: 60

Sample 10
Actual Age: 64
Adjusted Predicted Age: 54



In [ ]:
# Save the model
model.save("age_prediction_model.h5")

# Print confirmation message
print("Model saved successfully!")


Model saved successfully!


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
